<a href="https://colab.research.google.com/github/shivam110601/ai-video-captioning/blob/main/AI_Video_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [ ]:
!pip install --upgrade pip
!pip install transformers datasets[audio] accelerate torch ffmpeg-python moviepy gradio

In [ ]:
!apt-get install -y imagemagick

In [ ]:
!cp /etc/ImageMagick-6/policy.xml /etc/ImageMagick-6/policy.xml.bak
!sed -i 's/rights="none"/rights="read | write"/' /etc/ImageMagick-6/policy.xml

In [ ]:
from moviepy.config import change_settings
change_settings({"IMAGEMAGICK_BINARY": "/usr/bin/convert"})

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip
import ffmpeg
import gradio as gr

# whisper-small

In [ ]:
# Load the Whisper model and processor
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [ ]:
model_id = "openai/whisper-small"

In [ ]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,
    batch_size=1,
    torch_dtype=torch_dtype,
    device=device,
)

# Trancript generation

In [ ]:
# Function to extract audio from video
def extract_audio(video_path, output_audio_path="temp_audio.wav"):
    ffmpeg.input(video_path).output(output_audio_path).run(overwrite_output=True)
    return output_audio_path

In [ ]:
# Function to transcribe audio using Whisper
def transcribe_audio(audio_path):
    result = pipe(audio_path, return_timestamps='word', generate_kwargs={"language": "english"})  #currently word to word timestamp
    return result["chunks"]

# Ouput video with burned-in captin

In [ ]:
# Function to add captions to video
def add_captions_to_video(video_path, transcriptions, output_path="output_video.mp4"):
    video = VideoFileClip(video_path)
    subs = []

    for transcription in transcriptions:
        start, end = transcription['timestamp'][0], transcription['timestamp'][1]
        text = transcription["text"]

        txt_clip = (TextClip(text, font = 'Noto-Sans-Mono-Bold', fontsize=50, color='white', stroke_width=1, stroke_color='black')
                    .set_position(("center",0.85), relative=True)
                    .set_duration(end - start)
                    .set_start(start))

        subs.append(txt_clip)

    video_with_subs = CompositeVideoClip([video] + subs)
    video_with_subs.write_videofile(output_path, codec="libx264", audio_codec="aac")

    return output_path

# Gradio Application

In [ ]:
def process_video(video_path):
    audio_path = extract_audio(video_path)
    transcriptions = transcribe_audio(audio_path)
    output_video_path = add_captions_to_video(video_path, transcriptions)

    return output_video_path

In [ ]:
# Gradio interface
iface = gr.Interface(fn=process_video, inputs=gr.Video(), outputs="video", title="AI video captioning")

iface.launch(debug=True)

In [ ]:
iface.close()

Closing server running on port: 7860
